In [ ]:
# 3/3/23 update
# this notebook automatically loads flight level data, finds the median time, and pulls the SHIPS data
# closest to that time! it then saves important quantities in a text file!!

In [85]:
# import...
import numpy as np
import os
import matplotlib.pyplot as plt
import xarray as xr
import warnings
import sys
import pandas as pd
os.chdir(  "/Users/etmu9498/research/code/scripts")
import make_plots
import helper_fns



In [98]:
# load fl names for processed datasets (for 2021 as an example)
year = '2021'
fl_path = "/Users/etmu9498/research/data/in-situ-noaa-processed/" + year
names2021 = make_plots.load_flight_level( fl_path, print_files=False)

# choose a name and load the data
fl_name = '20210925H1_sam_processed.nc' # '20210813I1_grace_processed.nc' # names2021[ 21]
print( fl_name)
os.chdir( fl_path)
data = xr.open_dataset( fl_name)


20210925H1_sam_processed.nc


In [102]:
# find the median time for this dataset
print( 'first time: ' + str( data.time[0].values))
print( 'last time: ' + str( data.time[-1].values))
median = data.time[0].values + ( data.time[-1].values - data.time[0].values) / 2
print( 'midpoint: ' + str( median))

# find the closest 6 hour SHIPS interval to this time
hours = np.array( [0, 6, 12, 18, 24, 30]) # add 24 and 30 hours as wraparound cases
closetime = hours[ np.argmin( np.abs( hours - median)) ]

# roll back wraparound cases to the correct time (no 24 or 30 h cases in SHIPS database!)
wraparound = False
if closetime > 18:
    wraparound = True
    closetime -= 24

# print out useful metadata for this case!
strdate = fl_name[4:8] # original date: correct if wraparound time selected
# get the formatting correct
strtime = str( closetime)
if strtime == '0' or strtime == '6':
    strtime = '0' + strtime
strname = fl_name[ 11:15].upper()
# cut off the string at the underscore! allow for 2 and 3 character tc names
strtemp = ''
for chari, charval in enumerate( strname):
    if charval == '_':
        break
    else:
        strtemp += charval
strname = strtemp

print( 'name: ' + strname)
print( 'date: ' + strdate)
print( 'time: ' + strtime)
print( 'wraparound time? ' + str( wraparound))

first time: 17.186666666666667
last time: 27.145833333333336
midpoint: 22.16625
name: SAM
date: 0925
time: 00
wraparound time? True


In [103]:
# load the full ships dataset
os.chdir(  "/Users/etmu9498/research/data/ships/")
file1 = open('lsdiaga_1982_2021_sat_ts_5day.txt', 'r')
Lines = file1.readlines()
# there are a bunch of lines in the total ships file!!
print( len( Lines))



1884145


In [104]:
# save the header indices for 2021 grac data here for easy access later
header_inds = []
flind = 0

# go through all the lines
for ind in range( len( Lines)):
    # get the heading lines, and look for this TC's cases!
    if 'HEAD' and strname  in Lines[ ind]:

        
        # only keep 2021 cases
        if Lines[ ind][ 6 : 8] == str( year[ 2:4]):
            # print out the header and its index!
            # print( 'index = ' + str( ind))
            # print( Lines[ ind])
        
            # save this index in a separate list! add all indices for this tc + year combo, useful for wraparound times
            header_inds.append( ind)

            # find the cases with the valid date and time!
            if strdate + ' ' + strtime in Lines[ ind]:
                # no matter the case, find the index!
                flind = ind
            
# account for wraparound cases!
# wraparound cases: a bit annoying. Skip ahead 24 hours to the next date!
# this work
if wraparound:
    # index in the limited header_inds list of the header of interest! 
    small_list_ind = header_inds.index( flind)
    # shift 24 hours (6h * 4) ahead in indices
    flind = header_inds[ small_list_ind + 4]


# print results!
print( header_inds)
print( flind)
print( Lines[ flind])

# add the correct date (wraparounds accounted for)
correct_date = Lines[ flind][ 6 : 12]
print( correct_date)

[1869133, 1869272, 1869411, 1869550, 1869689, 1869828, 1869967, 1870106, 1870245, 1870384, 1870523, 1870662, 1870801, 1870940, 1871079, 1871218, 1871357, 1871496, 1871635, 1871774, 1871913, 1872052, 1872191, 1872330, 1872469, 1872608, 1872747, 1872886, 1873025, 1873164, 1873303, 1873442, 1873581, 1873720, 1873859, 1873998, 1874137, 1874276, 1874415, 1874554, 1874693, 1874832, 1874971, 1875110, 1875249, 1875388, 1875527, 1875666, 1875805, 1875944]
1870940
 SAM  210926 00  130   13.4   48.8  943 AL182021                                                                    HEAD                    

210926


In [111]:
# print all fields for current time!
# current time
i0 = flind
i1 = header_inds[ header_inds.index( flind) + 1]
for i in range( i0,  i1 ):
    print( Lines[ i])

 SAM  210926 00  130   13.4   48.8  943 AL182021                                                                    HEAD                    

  -12   -6    0    6   12   18   24   30   36   42   48   54   60   66   72   78   84   90   96  102  108  114  120 TIME                    

  105  115  130  130  130  135  130  115  105  105  105  115  115  115  115  115  115  115  125  125  125  125  125 VMAX                    

  960  950  943  943  939  927  940  948  955  958  955  952  952  947  947  952  952  945  940  937  937  941  938 MSLP                    

    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1    1 TYPE                    

             14   14   14   12   11   11   10    9    9    8    6    5    5    5    4    4    3    3    2    2    1 HIST                    

   25   15    0    0    0    5    0  -15  -25  -25  -25  -15  -15  -15  -15  -15  -15  -15   -5   -5   -5   -5   -5 DELV                    

   10 

In [106]:
# get 24h old and current vmax values
vmaxold = 0
vmax = 0
# 24h before
# make sure that there's SHIPS data 24h before the flight! if not, we can't get the old intensity
# idea: maybe when there's no - 24h data, but there is - 12h data, use the -12h column in that dataset! SHIPS usually has
# that data saved :)
if header_inds.index( flind) - 4 >= 0:
    header_ind = header_inds[ header_inds.index( flind) - 4]
    # only look in this date's variables!
    i0 = header_ind
    i1 = header_inds[ header_inds.index( flind) - 3]
    # search for vmax!
    for i in range( i0,  i1 ):
        if 'VMAX' in Lines[i]:
            vmaxold = Lines[i][12 : 15] # the last 3 vals for vmax at 0 hours
            break
else:
    print( "No data 24h before flight :/")
            
# current max values
i0 = flind
i1 = header_inds[ header_inds.index( flind) + 1]
for i in range( i0, i1):
    if 'VMAX' in Lines[i]:
        vmax = Lines[i][12 : 15] # the last 3 vals for vmax at 0 hours
        break

print( vmaxold)
print( vmax)

# pull previous and current shear dirs and mags
dirold = 0
direc = 0
magold = 0
mag = 0

# 24h before
if header_inds.index( flind) - 4 >= 0:
    i0 = header_inds[ header_inds.index( flind) - 4]
    i1 = header_inds[ header_inds.index( flind) - 3]
    # search for vmax!
    for i in range( i0,  i1 ):
        
        if i == i0 or i == i0 + 1:
            print( Lines[ i])
        
        if 'SHRD' in Lines[i]:
            print( Lines[ i])
            magold = Lines[i][11 : 15] # the last 4 vals for shrd at 0 hours
        if 'SHTD' in Lines[i]:
            print( Lines[ i])
            dirold = Lines[i][11 : 15] # the last 4 vals for shtd at 0 hours
else:
    print( "No data 24h before flight :/")

# current time
i0 = flind
i1 = header_inds[ header_inds.index( flind) + 1]
# search for current shear magnitudes and directions!
for i in range( i0,  i1 ):
    if i == i0 or i == i0 + 1:
        print( Lines[ i])

    if 'SHRD' in Lines[i]:
        print( Lines[ i])
        mag = Lines[i][11 : 15] # the last 4 vals for shrd at 0 hours
    if 'SHTD' in Lines[i]:
        print( Lines[ i])
        direc = Lines[i][11 : 15] # the last 4 vals for shtd at 0 hours


# do this for nice formatting! it gets rid of the extra whitespace :)
vmaxold = str( int( vmaxold))
vmax = str( int( vmax))
dirold = str( int( dirold))
direc = str( int( direc))
magold = str( np.round( float( magold ) / 10, 2)) # divide by 10 to get to kts, see SHIPS documentation
mag = str(  np.round( float( mag ) / 10, 2))

print( dirold)
print( magold)
print( direc)
print( mag)

 85
130
 SAM  210925 00   85   12.3   45.4  982 AL182021                                                                    HEAD                    

  -12   -6    0    6   12   18   24   30   36   42   48   54   60   66   72   78   84   90   96  102  108  114  120 TIME                    

             50   71  102   93   94  114   95   90  111  111   97   64   83  127  115  104  108  116  136  101  101 SHRD                    

             66   73   79   86   67   74   77   65   72   71   84   73   56   67   85   89   87   89   92  101   96 SHTD                    

 SAM  210926 00  130   13.4   48.8  943 AL182021                                                                    HEAD                    

  -12   -6    0    6   12   18   24   30   36   42   48   54   60   66   72   78   84   90   96  102  108  114  120 TIME                    

             86   98   95   86  103  109   94   62   86  122  119   97  107  105  127   93   86   99  109   69   78 SHRD                    

In [107]:
# new new and previous vmax, shear, and shear dir variables to a text document!

# get to the correct folder
os.chdir("/Users/etmu9498/research/data/ships-output/" )
# make sure the folder exists! if not, create it!
output_folder = year
if not os.path.isdir( output_folder):
    os.makedirs( output_folder)
    print( 'New folder created: ' + output_folder)
else:
    print( 'Folder ' + output_folder + ' already exists')
os.chdir("/Users/etmu9498/research/data/ships-output/" + year)

# create the file
file1 = open( fl_name[ : - 13] + '.txt', 'w+')

# add the metadata!
addlines = [ "SHIPS Date:\n", correct_date + '\n', "SHIPS Time (Hours, UTC):\n", strtime + '\n', "24h old vmax (kt):\n", 
            vmaxold + '\n', "Current vmax (kt):\n", vmax + '\n', "24h old shear direction (degrees):\n", dirold + '\n', 
            'Current shear direction (degrees):\n', direc + '\n', '24h old shear magnitude (kt):\n', magold + '\n', 
            'Current shear magnitude (kt):\n', mag + '\n']
file1.writelines( addlines)
file1.close()


Folder 2021 already exists
